In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('pivotable_cross.csv')

In [3]:
df.head()

,ID,Name,Type,Department,Store,Quantity,Price per Unit
0,1,Apples,Fruit,Produce,Philadephia,100,$0.50
1,2,Oranges,Fruit,Produce,Philadephia,100,$0.60
2,3,Banana,Fruit,Produce,Philadephia,100,$0.30
3,4,Lettuce,Vegetable,Produce,Philadephia,150,$0.20
4,5,Carrots,Vegetable,Produce,Philadephia,150,$0.30


In [5]:
df['Price per Unit'] = df['Price per Unit'].replace('\$', '', regex=True)
df['Price per Unit'] = pd.to_numeric(df['Price per Unit'])
df['Value'] = df['Quantity']*df['Price per Unit']
df.head()

,ID,Name,Type,Department,Store,Quantity,Price per Unit,Value
0,1,Apples,Fruit,Produce,Philadephia,100,0.5,50.0
1,2,Oranges,Fruit,Produce,Philadephia,100,0.6,60.0
2,3,Banana,Fruit,Produce,Philadephia,100,0.3,30.0
3,4,Lettuce,Vegetable,Produce,Philadephia,150,0.2,30.0
4,5,Carrots,Vegetable,Produce,Philadephia,150,0.3,45.0


There are multiple Pandas methods to aggregate and summarize the data above, two of which include the groupby method and pivot_table method. Please note that Pandas does have a pivot method, but this is different than pivot table in that it does not include an aggregate method argument.

For our exercise we will be using pivot_table over groupby; similar functions can be developed with groupby but pivot_table natively returns another data frame — making it easy to export to Excel or other spreadsheet tools.

In [6]:
df.pivot_table(values='Value',index=['Store','Department','Type'],aggfunc='sum')

Value
Store         Department Type             
New York City Bakery     Bread       658.5
                         Desert     1033.5
              Butcher    Fish        487.5
                         Meat        375.0
              Produce    Fruit       173.7
                         Vegetable   180.0
Philadephia   Bakery     Bread       222.0
                         Desert      910.0
              Butcher    Fish        320.0
                         Meat        240.0
              Produce    Fruit       140.0
                         Vegetable   135.0

The only built-in argument that helps us is “margins”, but implementing this argument only adds a final total, titled “All”.

In [7]:
df.pivot_table(values='Value',index=['Store','Department','Type'],aggfunc='sum',margins=True)

Value
Store         Department Type             
New York City Bakery     Bread       658.5
                         Desert     1033.5
              Butcher    Fish        487.5
                         Meat        375.0
              Produce    Fruit       173.7
                         Vegetable   180.0
Philadephia   Bakery     Bread       222.0
                         Desert      910.0
              Butcher    Fish        320.0
                         Meat        240.0
              Produce    Fruit       140.0
                         Vegetable   135.0
All                                 4875.2

In order to add subtotals for our stores as well as departments, we will need to construct a loop that will create a pivot_table repeatedly, with each iteration adding another level of indices to tabulate — then concatenate all of the data frames together. Ideally, this would also occur dynamically to accommodate future datasets with more levels of indices or other columns. The relevant function is below.

In [10]:
def pivot_table_w_subtotals(df, values, indices, columns, aggfunc, fill_value):
    '''
    Adds tabulated subtotals to pandas pivot tables with multiple hierarchical indices.
    
    Args:
    - df - dataframe used in pivot table
    - values - values used to aggregrate
    - indices - ordered list of indices to aggregrate by
    - columns - columns to aggregrate by
    - aggfunc - function used to aggregrate (np.max, np.mean, np.sum, etc)
    - fill_value - value used to in place of empty cells
    
    Returns:
    -flat table with data aggregrated and tabulated
    
    '''
    listOfTable = []
    for indexNumber in range(len(indices)):
        n = indexNumber+1
        
        table = pd.pivot_table(df,values=values,index=indices[:n],columns=columns,aggfunc=aggfunc,fill_value=fill_value).reset_index()
        for column in indices[n:]:
            table[column] = ''
            
        listOfTable.append(table)
    concatTable = pd.concat(listOfTable).sort_index()
    concatTable = concatTable.set_index(keys=indices)
    return concatTable.sort_index(axis=0,ascending=True)

In [11]:
pivot_table_w_subtotals(df=df,values='Value',indices=['Store','Department','Type'],columns=[],aggfunc='sum',fill_value='')

Value
Store         Department Type             
New York City                       2908.2
              Bakery                1692.0
                         Bread       658.5
                         Desert     1033.5
              Butcher                862.5
                         Fish        487.5
                         Meat        375.0
              Produce                353.7
                         Fruit       173.7
                         Vegetable   180.0
Philadephia                         1967.0
              Bakery                1132.0
                         Bread       222.0
                         Desert      910.0
              Butcher                560.0
                         Fish        320.0
                         Meat        240.0
              Produce                275.0
                         Fruit       140.0
                         Vegetable   135.0

Notice that now we have subtotals by store and by department! We are now just missing the grand total, but we can add this by using the “margins” function argument on only one iteration.

In [12]:
def pivot_table_w_subtotals(df, values, indices, columns, aggfunc, fill_value):
    '''
    Adds tabulated subtotals to pandas pivot tables with multiple hierarchical indices.
    
    Args:
    - df - dataframe used in pivot table
    - values - values used to aggregrate
    - indices - ordered list of indices to aggregrate by
    - columns - columns to aggregrate by
    - aggfunc - function used to aggregrate (np.max, np.mean, np.sum, etc)
    - fill_value - value used to in place of empty cells
    
    Returns:
    -flat table with data aggregrated and tabulated
    
    '''
    listOfTable = []
    for indexNumber in range(len(indices)):
        n = indexNumber+1
        if n == 1:
            table = pd.pivot_table(df,values=values,index=indices[:n],columns=columns,aggfunc=aggfunc,fill_value=fill_value,margins=True)
        else:
            table = pd.pivot_table(df,values=values,index=indices[:n],columns=columns,aggfunc=aggfunc,fill_value=fill_value)
        table = table.reset_index()
        for column in indices[n:]:
            table[column] = ''
        listOfTable.append(table)
    concatTable = pd.concat(listOfTable).sort_index()
    concatTable = concatTable.set_index(keys=indices)
    return concatTable.sort_index(axis=0,ascending=True)
pivot_table_w_subtotals(df=df,values='Value',indices=['Store','Department','Type'],columns=[],aggfunc='sum',fill_value='')

Value
Store         Department Type             
All                                 4875.2
New York City                       2908.2
              Bakery                1692.0
                         Bread       658.5
                         Desert     1033.5
              Butcher                862.5
                         Fish        487.5
                         Meat        375.0
              Produce                353.7
                         Fruit       173.7
                         Vegetable   180.0
Philadephia                         1967.0
              Bakery                1132.0
                         Bread       222.0
                         Desert      910.0
              Butcher                560.0
                         Fish        320.0
                         Meat        240.0
              Produce                275.0
                         Fruit       140.0
                         Vegetable   135.0